In [ ]:
import pandas as pd
import phoenix as px
from pandas.testing import assert_frame_equal
from phoenix.db import models
from sqlalchemy import and_, create_engine, select
from sqlalchemy.orm import aliased, sessionmaker

In [ ]:
PostgresSession = sessionmaker(
    create_engine(
        "postgresql+psycopg://localhost:5432/postgres?user=postgres&password=mysecretpassword",
        echo=True,
    ),
    expire_on_commit=False,
)
SqliteSession = sessionmaker(
    create_engine("sqlite:////Users/xandersong/.phoenix/phoenix.db", echo=True),
    expire_on_commit=False,
)

In [ ]:
original_endpoint = "http://127.0.0.1:6008"
sqlite_endpoint = "http://127.0.0.1:6006"
postgres_endpoint = "http://127.0.0.1:6007"

In [ ]:
# SELECT * FROM spans
# JOIN (
#     SELECT spans.id, sa.score, sa.label FROM spans
#     JOIN span_annotations sa on spans.id = sa.span_rowid
# ) B ON spans.id == B.id
# WHERE B.score > 0.5 AND B.label == 'factual' AND spans.name == 'query';

In [ ]:
def get_spans_dataframe(endpoint: str, filter_condition: str):
    return (
        px.Client(endpoint=endpoint)
        .get_spans_dataframe(filter_condition)
        .sort_index()
        .reindex(sorted(sqlite_df.columns), axis=1)
    )

In [ ]:
filter_condition = "evals['Q&A Correctness'].label == 'correct'"
original_df = get_spans_dataframe(endpoint=original_endpoint, filter_condition=filter_condition)
postgres_df = get_spans_dataframe(endpoint=postgres_endpoint, filter_condition=filter_condition)
sqlite_df = get_spans_dataframe(endpoint=sqlite_endpoint, filter_condition=filter_condition)

In [ ]:
original_df.compare(sqlite_df, result_names=("original", "sqlite"))

In [ ]:
postgres_df

In [ ]:
original_df.compare(postgres_df, result_names=("original", "postgres"))

In [ ]:
sqlite_df[["attributes.openinference.span.kind", "events", "parent_id"]]

In [ ]:
original_df[["attributes.openinference.span.kind", "events", "parent_id"]]

In [ ]:
original_df["events"].map(len).value_counts()

In [ ]:
type(sqlite_df["events"].iloc[0])

In [ ]:
original_df.equals(sqlite_df)

In [ ]:
sqlite_df["parent_id"].isna().sum()

In [ ]:
original_df["attributes.openinference.span.kind"]

In [ ]:
original_df.columns

In [ ]:
sqlite_df.columns

In [ ]:
stmt = (
    select(models.Span.span_id)
    .join(models.SpanAnnotation)
    .where(
        and_(
            models.SpanAnnotation.name == "Q&A Correctness",
            models.SpanAnnotation.label == "correct",
        ),
    )
)
with SqliteSession() as sqlite_session:
    sqlite_df = pd.read_sql(
        stmt,
        sqlite_session.connection(),
        index_col="span_id",
    )
with PostgresSession() as postgres_session:
    postgres_df = pd.read_sql(
        stmt,
        postgres_session.connection(),
        index_col="span_id",
    )
assert_frame_equal(
    sqlite_df.sort_index().sort_index(axis=1),
    postgres_df.sort_index().sort_index(axis=1),
)

```
evals["Q&A Correctness"].label == "correct" and evals["Hallucination"].label == "hallucinated"
```

In [ ]:
A = aliased(models.SpanAnnotation, name="first-table")
B = aliased(models.SpanAnnotation)
stmt = (
    select(models.Span.span_id)
    .join(A, onclause=A.name == "Q&A Correctness")
    .join(B, onclause=B.name == "Hallucination")
    .where(A.label == "correct")
    .where(B.label == "hallucinated")
)
with SqliteSession() as sqlite_session:
    sqlite_df = pd.read_sql(
        stmt,
        sqlite_session.connection(),
        index_col="span_id",
    )
with PostgresSession() as postgres_session:
    postgres_df = pd.read_sql(
        stmt,
        postgres_session.connection(),
        index_col="span_id",
    )
assert_frame_equal(
    sqlite_df.sort_index().sort_index(axis=1),
    postgres_df.sort_index().sort_index(axis=1),
)

In [ ]:
from sqlalchemy import inspect
from sqlalchemy.orm import aliased

insp = inspect(A)

In [ ]:
from sqlalchemy.orm.util import AliasedClass

isinstance(A, AliasedClass)

In [ ]:
from sqlalchemy.sql.roles import JoinTargetRole

isinstance(A, JoinTargetRole)

In [ ]:
insp.name

In [ ]:
# SELECT span_annotations.span_rowid,
# MAX(CASE WHEN name = 'Hallucination' and score = 0 THEN 1 ELSE 0 END) AS A,
# MAX(CASE WHEN name = 'Q&A Correctness' and label = 'correct' THEN 1 ELSE 0 END) AS B
# FROM span_annotations
# WHERE name in ('Hallucination', 'Q&A Correctness')
# GROUP BY span_annotations.span_rowid
# HAVING A = 1 and B = 1
# ORDER BY span_rowid;